### Installing and Importing Required Libraries

We begin by installing all the libraries needed for our protein function prediction project. This includes tools for biological data handling (`biopython`, `goatools`), machine learning (`scikit-learn`, `scikit-multilearn`), data manipulation (`pandas`, `numpy`), and visualization (`matplotlib`).

After installation, we import these libraries to make them available in our environment. This setup ensures we can preprocess biological data, perform multi-label classification, manage the Gene Ontology hierarchy, and evaluate our models using metrics like F1-score and precision. We've also included utilities like `os` to handle file paths when needed.


In [ ]:
# ✅ Install all required libraries
!pip install biopython
!pip install goatools
!pip install scikit-learn
!pip install scikit-multilearn
!pip install pandas
!pip install numpy
!pip install matplotlib  # Optional, for any future plotting

# ✅ Now you can safely import everything used in your notebook
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt  # Optional if needed

from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.decomposition import PCA  # If used later
from sklearn.metrics import f1_score, precision_score  # For evaluation

from skmultilearn.model_selection import iterative_train_test_split

from Bio import SeqIO
from goatools.obo_parser import GODag

import os  # For managing paths if needed


In [2]:
# We load the embeddings from a CSV file
embeddings = pd.read_csv('embeddings.csv')


###  Loading Protein Sequences, Annotations, and GO Hierarchy

We load three essential datasets required for our project:

1. **Protein Sequences**: Using `Bio.SeqIO`, we parse the `train_sequences.fasta` file and store each protein’s ID and its amino acid sequence in a dictionary called `train_sequences`.

2. **GO Annotations**: We read the `train_terms.tsv` file into a DataFrame called `train_terms`. This contains mappings between protein IDs and their associated Gene Ontology (GO) terms.

3. **GO Hierarchy**: Using `goatools`, we parse the `go-basic.obo` file with `GODag`, which gives us access to the hierarchical structure of GO terms. This will be useful later when incorporating ontology-aware strategies like ancestor term propagation.


In [2]:
# Load train_sequences.fasta
from Bio import SeqIO
train_sequences = {}
for record in SeqIO.parse(r'C:\Users\Utente\OneDrive - uniroma1.it\Esami\Statistical Learning\Project\Implementation\Train\train_sequences.fasta', 'fasta'):
    train_sequences[record.id] = str(record.seq)
# Load train_terms.tsv
train_terms = pd.read_csv(r'C:\Users\Utente\OneDrive - uniroma1.it\Esami\Statistical Learning\Project\Implementation\Train\train_terms.tsv', sep='\t')
# Load go-basic.obo
from goatools.obo_parser import GODag
go_dag = GODag(r'C:\Users\Utente\OneDrive - uniroma1.it\Esami\Statistical Learning\Project\Implementation\Train\go-basic.obo')

C:\Users\Utente\OneDrive - uniroma1.it\Esami\Statistical Learning\Project\Implementation\Train\go-basic.obo: fmt(1.2) rel(2023-01-01) 46,739 Terms


###  Building the Target Label Matrix

We construct the binary label matrix needed for multi-label classification by following these steps:

1. **Top GO Terms Selection**: We focus on the 1500 most frequent GO terms across all proteins, helping reduce label sparsity and memory usage.

2. **Filtering Proteins**: We ensure we only retain annotations for proteins that have corresponding embeddings available.

3. **Term Grouping**: For each protein, we collect its list of GO terms and fill in empty lists for proteins with no annotations in the top 1500 terms.

4. **Binarization**: Using `MultiLabelBinarizer`, we convert each protein’s list of GO terms into a binary row vector — 1 if the term is present, 0 otherwise.

5. **Final Label Matrix**: We assemble everything into a clean DataFrame (`label_df`) with one row per protein and one column per GO term. This matrix is then saved for reuse.

This matrix will serve as our target `Y` for training models and evaluating predictions.


In [3]:
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer


k = 1500  # Number of top GO terms to keep

embedding_df = pd.read_csv("embeddings.csv")
protein_ids = embedding_df["Protein Id"].tolist()

# === Step 2: Load GO Annotations ===
terms_df = pd.read_csv("Train/train_terms.tsv", sep='\t')

# === Step 3: Filter to proteins that have embeddings ===
terms_df = terms_df[terms_df['EntryID'].isin(protein_ids)]

# === Step 4: Keep only the top 1500 GO terms based on frequency ===
top_terms = terms_df['term'].value_counts().nlargest(k).index
terms_df = terms_df[terms_df['term'].isin(top_terms)]

# === Step 5: Group GO terms by protein ===
grouped_terms = terms_df.groupby("EntryID")["term"].apply(list)

# === Step 6: Fill missing proteins with empty lists ===
go_terms_list = [grouped_terms.get(pid, []) for pid in protein_ids]

# === Step 7: Create binary matrix ===
mlb = MultiLabelBinarizer(classes=sorted(top_terms))
Y = mlb.fit_transform(go_terms_list)
go_terms = mlb.classes_

# === Step 8: Turn into a DataFrame ===
label_df = pd.DataFrame(Y, columns=go_terms)
label_df.insert(0, "Protein Id", protein_ids)

# === (Optional) Save for reuse ===
label_df.to_csv("label_matrix_1500.csv", index=False)

print(f"✅ Label matrix shape: {label_df.shape}")


✅ Label matrix shape: (142246, 1501)


###  Retrieving GO Terms for a Protein

We define a helper function `get_go_terms(protein_id)` to easily retrieve the GO terms associated with a given protein:

- It searches the `label_df` (our binary label matrix) for the specified `protein_id`.
- If the protein exists, it extracts all the GO terms where the corresponding value is 1.
- These GO terms are returned as a list.
- If the protein is not found, the function returns `None`.

This is particularly useful for debugging, manual verification, or evaluating the model’s output for specific proteins.


In [5]:
def get_go_terms(protein_id):
    """
    Given a protein ID, return the corresponding GO terms.
    """
    if protein_id in label_df["Protein Id"].values:
        # Get the row corresponding to the protein ID
        row = label_df[label_df["Protein Id"] == protein_id].iloc[:, 1:]
        # Extract the GO terms where the value is 1
        go_terms = row.columns[row.values.flatten() == 1].tolist()
        return go_terms
    else:
        return None

# Example usage
protein_id = "P20536"  # Replace with a valid protein ID from your dataset
go_terms = get_go_terms(protein_id)
print(f"GO terms for protein {protein_id}: {go_terms}")

GO terms for protein P20536: ['GO:0003674', 'GO:0005488', 'GO:0005515', 'GO:0006139', 'GO:0006259', 'GO:0006725', 'GO:0006807', 'GO:0008150', 'GO:0008152', 'GO:0009058', 'GO:0009059', 'GO:0009987', 'GO:0016032', 'GO:0018130', 'GO:0019058', 'GO:0019438', 'GO:0034641', 'GO:0034654', 'GO:0043170', 'GO:0044237', 'GO:0044238', 'GO:0044249', 'GO:0044260', 'GO:0044271', 'GO:0046483', 'GO:0071704', 'GO:0090304', 'GO:1901360', 'GO:1901362', 'GO:1901576']


### Sanity Check on the Label Matrix

We randomly sample 500 proteins and verify that their GO term annotations in the label matrix (`label_df`) exactly match the original annotations from `terms_df`, after filtering to the top 1500 GO terms.

For each sampled protein:
- We extract the original terms from `terms_df`.
- We retrieve the binarized labels from `label_df`.
- We compare the two lists to confirm correctness.

Finally, we check if all 500 comparisons are consistent. If `all(is_true)` returns `True`, we can be confident that our label matrix was constructed correctly.


In [6]:
# Pick a few proteins to manually check
sample_ids = label_df["Protein Id"].sample(500, random_state=42)
is_true = []

for pid in sample_ids:
    original_terms = terms_df[terms_df["EntryID"] == pid]["term"].tolist()
    original_terms = [t for t in original_terms if t in go_terms]  # Filter to top 1500
    predicted_terms = label_df[label_df["Protein Id"] == pid].iloc[:, 1:]
    predicted_terms = predicted_terms.loc[:, (predicted_terms == 1).any(axis=0)].columns.tolist()

    #print(f"🔍 Protein {pid}")
    #print(f"From file: {sorted(original_terms)}")
    #print(f"From matrix: {sorted(predicted_terms)}")
    #print(sorted(original_terms) == sorted(predicted_terms))
    is_true.append(sorted(original_terms) == sorted(predicted_terms))
    #print("---")
all(is_true)  # Check if all are True

False

### Splitting the Dataset into Train, Validation, and Test Sets

In this cell, we randomly split our dataset of protein embeddings and GO labels into training, validation, and test sets:

1. **Extract Input and Labels**:  
   We extract the input features `X` (embeddings), target labels `Y` (GO terms), and corresponding protein IDs from the full dataset.

2. **Initial Test Split (10%)**:  
   We use `train_test_split` to separate 10% of the data as the test set. This split is done **without stratification**.

3. **Train/Validation Split (from remaining 90%)**:  
   From the remaining data, we further split 10% for validation, resulting in approximately 80% train, 10% val, 10% test overall.

4. **Check Final Shapes**:  
   We print the final sizes of each split to confirm the partitioning.

This simple approach is fast and works well when stratification is not critical or causes issues due to label imbalance.


In [7]:
from sklearn.model_selection import train_test_split
import numpy as np

# Step 1: Prepare X, Y
X = embedding_df.drop("Protein Id", axis=1).values
Y = label_df.drop("Protein Id", axis=1).values
protein_ids = np.array(embedding_df["Protein Id"])

# Step 2: Simple random split (no stratify)
X_remain, X_test, y_remain, y_test, id_remain, id_test = train_test_split(
    X, Y, protein_ids, test_size=0.10, random_state=42
)

# Step 3: Split train/val
X_train, X_val, y_train, y_val, id_train, id_val = train_test_split(
    X_remain, y_remain, id_remain, test_size=1/9, random_state=42
)

# Step 4: Print shapes
print("✅ Final Split:")
print(f"Train:      {X_train.shape}, {y_train.shape}")
print(f"Validation: {X_val.shape}, {y_val.shape}")
print(f"Test:       {X_test.shape}, {y_test.shape}")


✅ Final Split:
Train:      (113796, 1024), (113796, 1500)
Validation: (14225, 1024), (14225, 1500)
Test:       (14225, 1024), (14225, 1500)


In [8]:
# Save embeddings
pd.DataFrame(X_train).to_csv("X_train.csv", index=False)
pd.DataFrame(X_val).to_csv("X_val.csv", index=False)
pd.DataFrame(X_test).to_csv("X_test.csv", index=False)

# Save labels
pd.DataFrame(y_train, columns=label_df.columns[1:]).to_csv("y_train.csv", index=False)
pd.DataFrame(y_val, columns=label_df.columns[1:]).to_csv("y_val.csv", index=False)
pd.DataFrame(y_test, columns=label_df.columns[1:]).to_csv("y_test.csv", index=False)